# 05 Tooling | 01 Semantic Kernel | 03 - Agent Group Chat

## Step 1: Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

This notebooks highlights how Semantic Kernel simplifies the process of storing and retrieving vector data. 


In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.38.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";


Console.WriteLine($"Configuration loaded...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.38.0 Microsoft.SemanticKernel.Agents.Core, 1.38.0-alpha Microsoft.SemanticKernel.Connectors.OpenAI, 1.38.0

Configuration loaded...


## Step 2: Plugin Definition

The agent chat will use Semantic Kernel plugins to access simulated private data

In [ ]:
public class RetrieveWinner
{
    [KernelFunction("get_sport_event_winner")]
    //Description is just necessary if function name is not self-explanatory
    [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the winner of the sport event.
        return "Kansas City Chiefs";
    }
}

public class RetrieveScore
{
    [KernelFunction("get_sport_event_score")]
    //Description is just necessary if function name is cryptic and not self-explanatory
    [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the result of the sport event.
        return "25:22";
    }
}

Console.WriteLine($"Plugins defined...")

Plugins defined...


## Step 3: Agent Definition

Two agents who will communicate to solve a specific task are created:

- RetrieveWinner agent: tasked to retrieve the winner of a sport event
- RetrieveScore agent: tasked to retrieve the score of a sport event

Both agents use the provided plug-ins to solve the provided task.


In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001

//Define RetrieveWinner Agent
IKernelBuilder retrieveWinnerKernelBuilder = Kernel.CreateBuilder();
retrieveWinnerKernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: oAiApiKey, 
    endpoint: oAiEndpoint, 
    deploymentName: chatCompletionDeploymentName
);
retrieveWinnerKernelBuilder.Plugins.AddFromType<RetrieveWinner>("RetrieveWinner");
Kernel retrieveWinnerKernel = retrieveWinnerKernelBuilder.Build();

string agentName = "RetrieveWinner";
string agentInstruction = "You can retrieve the winner of a sport event";

ChatCompletionAgent retrieveWinnerAgent = new ChatCompletionAgent()
{
    Name = agentName,
    Instructions = agentInstruction,
    Kernel = retrieveWinnerKernel,
    Arguments = new KernelArguments(
        new OpenAIPromptExecutionSettings() { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        }
    )
};

//Define retrieveScore Agent
IKernelBuilder retrieveScoreKernelBuilder = Kernel.CreateBuilder();
retrieveScoreKernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: oAiApiKey, 
    endpoint: oAiEndpoint, 
    deploymentName: chatCompletionDeploymentName
);
retrieveScoreKernelBuilder.Plugins.AddFromType<RetrieveScore>("RetrieveScore");
Kernel retrieveScoreKernel = retrieveScoreKernelBuilder.Build();

agentName = "RetrieveScore";
agentInstruction = "You can retrieve the final score of a sport event";

ChatCompletionAgent retrieveScoreAgent = new ChatCompletionAgent()
{
    Name = agentName,
    Instructions = agentInstruction,
    Kernel = retrieveScoreKernel,
    Arguments = new KernelArguments(
        new OpenAIPromptExecutionSettings() { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        }
    )
};

Console.WriteLine($"retrieveWinner Agent created...");
Console.WriteLine($"retrieveScore Agent created...");

retrieveWinner Agent created...
retrieveScore Agent created...


## Step 4: Create Group Chat

An group chat is created with a custom termination condition. The agent group chat should terminate when there's a winning team and a score provided.

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001

KernelFunction terminationFunction = AgentGroupChat.CreatePromptFunctionForStrategy(
    $$$""" 
        Determine if there's a sport event and a score.
        Be precise. There needs to be a sport event and a score. 
        If there's no sport event or no score or both, respond with a single word: no
        If so, respond with a single word: yes

        History:
        {{$history}}
    """,
    safeParameterNames: "history"
);
KernelFunctionTerminationStrategy kernelFunctionTerminationStrategy = 
    new KernelFunctionTerminationStrategy(terminationFunction, retrieveScoreKernel){
        ResultParser = (result) => result.GetValue<string>()?.Contains("yes", StringComparison.OrdinalIgnoreCase) ?? false,
        HistoryVariableName = "history",
        MaximumIterations = 10

    };

AgentGroupChat agentGroupChat = new AgentGroupChat(retrieveWinnerAgent, retrieveScoreAgent);
agentGroupChat.ExecutionSettings = new AgentGroupChatSettings(){
    TerminationStrategy = kernelFunctionTerminationStrategy 
};

Console.WriteLine($"AgentGroupChat with custom termination criteria created...");

AgentGroupChat with custom termination criteria created...


## Step 5: Execute Group Chat

The group chat with the two defined agents will be executed.

In [ ]:
#pragma warning disable SKEXP0110

string prompt = "Who won the Super Bowl 2024?. Return name of the winning team and the score.";

agentGroupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, prompt));

string response = "";
await foreach (ChatMessageContent chatMessageContent in agentGroupChat.InvokeAsync())
{
    response = String.Concat(response, chatMessageContent.Content, "\n");
    Console.WriteLine(chatMessageContent.Content);
}

The Kansas City Chiefs won the Super Bowl in 2024. Unfortunately, the score is not available.
The Kansas City Chiefs won the Super Bowl 2024 with a score of 25-22.
